# Solana Data Explorer - Simplified Analysis

Simple analysis of wallet PnL and smart traders data.

## Data Sources
- **Silver Layer**: Wallet PnL metrics with FIFO methodology
- **Gold Layer**: Filtered smart traders from silver data

In [ ]:
import duckdb
import pandas as pd
import numpy as np
from datetime import datetime

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("📊 Solana Data Explorer - Ready for Analysis")

## 🔌 Database Connection

In [ ]:
# Connect to DuckDB with MinIO configuration
# Note: Running from outside Docker, so we create a local connection
conn = duckdb.connect()  # In-memory database since we're accessing S3 directly

# Install and load httpfs extension for S3 access
try:
    conn.execute("INSTALL httpfs;")
except:
    pass  # Already installed
    
conn.execute("LOAD httpfs;")

# Configure S3/MinIO access - use localhost since we're outside Docker
conn.execute("SET s3_endpoint='localhost:9000';")
conn.execute("SET s3_access_key_id='minioadmin';")
conn.execute("SET s3_secret_access_key='minioadmin123';")
conn.execute("SET s3_use_ssl=false;")
conn.execute("SET s3_url_style='path';")

print("✅ Connected to DuckDB with MinIO S3 access (localhost)")

## 📊 Silver Wallet PnL - Complete Analysis

In [ ]:
# Load ALL Silver Wallet PnL Data
print("📊 Loading ALL Silver Wallet PnL Records...")

try:
    silver_df = conn.execute("""
        SELECT * FROM read_parquet('s3://solana-data/silver/wallet_pnl/**/*.parquet')
        ORDER BY wallet_address, time_period
    """).df()
    
    print(f"✅ Loaded {len(silver_df):,} silver wallet PnL records")
    print(f"📊 Shape: {silver_df.shape}")
    print(f"🏦 Unique Wallets: {silver_df['wallet_address'].nunique():,}")
    print(f"⏰ Timeframes: {sorted(silver_df['time_period'].unique())}")
    
except Exception as e:
    print(f"❌ Error loading silver data: {e}")
    silver_df = pd.DataFrame()

In [ ]:
# Silver Data - Complete Column Statistics
if not silver_df.empty:
    print("🔍 SILVER WALLET PNL - COMPLETE COLUMN ANALYSIS")
    print("="*80)
    
    # Basic info
    print(f"\n📊 Dataset Shape: {silver_df.shape}")
    print(f"🏦 Unique Wallets: {silver_df['wallet_address'].nunique():,}")
    print(f"⏰ Timeframes: {sorted(silver_df['time_period'].unique())}")
    
    # Column data types
    print(f"\n📋 Column Data Types:")
    for col, dtype in silver_df.dtypes.items():
        null_count = silver_df[col].isnull().sum()
        null_pct = (null_count / len(silver_df)) * 100
        print(f"  {col:25} | {str(dtype):15} | Nulls: {null_count:,} ({null_pct:.1f}%)")
    
    # Numerical columns statistics
    print(f"\n📈 NUMERICAL COLUMNS - DETAILED STATISTICS:")
    numerical_cols = silver_df.select_dtypes(include=[np.number]).columns
    
    for col in numerical_cols:
        if col in silver_df.columns:
            print(f"\n  📊 {col.upper()}:")
            stats = silver_df[col].describe()
            print(f"     Count: {stats['count']:,.0f}")
            print(f"     Mean:  {stats['mean']:,.2f}")
            print(f"     Std:   {stats['std']:,.2f}")
            print(f"     Min:   {stats['min']:,.2f}")
            print(f"     25%:   {stats['25%']:,.2f}")
            print(f"     50%:   {stats['50%']:,.2f}")
            print(f"     75%:   {stats['75%']:,.2f}")
            print(f"     Max:   {stats['max']:,.2f}")
            
            # Additional insights
            positive_count = (silver_df[col] > 0).sum()
            negative_count = (silver_df[col] < 0).sum()
            zero_count = (silver_df[col] == 0).sum()
            
            print(f"     Positive: {positive_count:,} ({positive_count/len(silver_df)*100:.1f}%)")
            print(f"     Negative: {negative_count:,} ({negative_count/len(silver_df)*100:.1f}%)")
            print(f"     Zero:     {zero_count:,} ({zero_count/len(silver_df)*100:.1f}%)")
    
    # Categorical columns
    print(f"\n📝 CATEGORICAL COLUMNS:")
    categorical_cols = silver_df.select_dtypes(include=['object', 'string']).columns
    
    for col in categorical_cols:
        if col in silver_df.columns:
            unique_count = silver_df[col].nunique()
            print(f"\n  📊 {col.upper()}:")
            print(f"     Unique values: {unique_count:,}")
            
            if unique_count <= 20:  # Show value counts for small categories
                value_counts = silver_df[col].value_counts()
                for value, count in value_counts.items():
                    print(f"     {str(value):20} : {count:,} ({count/len(silver_df)*100:.1f}%)")
            else:
                print(f"     Top 5 values:")
                top_values = silver_df[col].value_counts().head()
                for value, count in top_values.items():
                    print(f"     {str(value)[:20]:20} : {count:,} ({count/len(silver_df)*100:.1f}%)")
    
else:
    print("❌ No silver data available for analysis")

## 🥇 Gold Smart Wallets - Complete Analysis

In [ ]:
# Load ALL Gold Smart Wallets Data
print("🥇 Loading ALL Gold Smart Wallets Records...")

# Try multiple sources to find the gold data
# Note: Since we're outside Docker, we can't access DuckDB tables directly
gold_sources = [
    "read_parquet('s3://solana-data/gold/smart_wallets/**/*.parquet')",  # Direct file
    "read_parquet('s3://webhook-data/gold/smart_wallets/**/*.parquet')"   # Webhook bucket
]

gold_df = pd.DataFrame()

for source in gold_sources:
    try:
        gold_df = conn.execute(f"SELECT * FROM {source} ORDER BY wallet_address").df()
        print(f"✅ Loaded {len(gold_df):,} gold records from: {source}")
        print(f"📊 Shape: {gold_df.shape}")
        print(f"🏦 Unique Wallets: {gold_df['wallet_address'].nunique():,}")
        if 'time_period' in gold_df.columns:
            print(f"⏰ Timeframes: {sorted(gold_df['time_period'].unique())}")
        break
    except Exception as e:
        print(f"❌ Failed to load from {source}: {e}")
        continue

if gold_df.empty:
    print("❌ No gold data found in any location")
    print("💡 Try running the DBT smart wallets DAG first to generate gold data")

In [ ]:
# Gold Data - Complete Column Statistics  
if not gold_df.empty:
    print("🔍 GOLD SMART WALLETS - COMPLETE COLUMN ANALYSIS")
    print("="*80)
    
    # Basic info
    print(f"\n📊 Dataset Shape: {gold_df.shape}")
    print(f"🏦 Unique Wallets: {gold_df['wallet_address'].nunique():,}")
    if 'time_period' in gold_df.columns:
        print(f"⏰ Timeframes: {sorted(gold_df['time_period'].unique())}")
    
    # Column data types
    print(f"\n📋 Column Data Types:")
    for col, dtype in gold_df.dtypes.items():
        null_count = gold_df[col].isnull().sum()
        null_pct = (null_count / len(gold_df)) * 100
        print(f"  {col:25} | {str(dtype):15} | Nulls: {null_count:,} ({null_pct:.1f}%)")
    
    # Numerical columns statistics
    print(f"\n📈 NUMERICAL COLUMNS - DETAILED STATISTICS:")
    numerical_cols = gold_df.select_dtypes(include=[np.number]).columns
    
    for col in numerical_cols:
        if col in gold_df.columns:
            print(f"\n  📊 {col.upper()}:")
            stats = gold_df[col].describe()
            print(f"     Count: {stats['count']:,.0f}")
            print(f"     Mean:  {stats['mean']:,.3f}")
            print(f"     Std:   {stats['std']:,.3f}")
            print(f"     Min:   {stats['min']:,.3f}")
            print(f"     25%:   {stats['25%']:,.3f}")
            print(f"     50%:   {stats['50%']:,.3f}")
            print(f"     75%:   {stats['75%']:,.3f}")
            print(f"     Max:   {stats['max']:,.3f}")
            
            # Additional insights for meaningful columns
            if col in ['smart_trader_score', 'trade_frequency_daily', 'win_rate']:
                positive_count = (gold_df[col] > 0).sum()
                negative_count = (gold_df[col] < 0).sum()
                zero_count = (gold_df[col] == 0).sum()
                
                print(f"     Positive: {positive_count:,} ({positive_count/len(gold_df)*100:.1f}%)")
                print(f"     Negative: {negative_count:,} ({negative_count/len(gold_df)*100:.1f}%)")
                print(f"     Zero:     {zero_count:,} ({zero_count/len(gold_df)*100:.1f}%)")
    
    # Categorical columns
    print(f"\n📝 CATEGORICAL COLUMNS:")
    categorical_cols = gold_df.select_dtypes(include=['object', 'string']).columns
    
    for col in categorical_cols:
        if col in gold_df.columns:
            unique_count = gold_df[col].nunique()
            print(f"\n  📊 {col.upper()}:")
            print(f"     Unique values: {unique_count:,}")
            
            # Always show value counts for categorical data
            value_counts = gold_df[col].value_counts()
            for value, count in value_counts.items():
                pct = (count / len(gold_df)) * 100
                print(f"     {str(value):25} : {count:,} ({pct:.1f}%)")
    
    # Sample records
    print(f"\n🎯 SAMPLE RECORDS (First 3):")
    print(gold_df.head(3).to_string())
    
else:
    print("❌ No gold data available for analysis")

## 🔄 Silver vs Gold Comparison

In [ ]:
# Compare Silver vs Gold layers
if not silver_df.empty and not gold_df.empty:
    print("🔄 SILVER vs GOLD LAYER COMPARISON")
    print("="*60)
    
    # Basic comparison
    silver_wallets = silver_df['wallet_address'].nunique()
    gold_wallets = gold_df['wallet_address'].nunique()
    
    print(f"📊 RECORD COUNTS:")
    print(f"  Silver Layer: {len(silver_df):,} records")
    print(f"  Gold Layer:   {len(gold_df):,} records")
    
    print(f"\n🏦 UNIQUE WALLETS:")
    print(f"  Silver Layer: {silver_wallets:,} wallets")
    print(f"  Gold Layer:   {gold_wallets:,} wallets")
    print(f"  Filtered Out: {silver_wallets - gold_wallets:,} wallets")
    print(f"  Filter Rate:  {((silver_wallets - gold_wallets)/silver_wallets*100):.1f}%")
    
    # Timeframe comparison
    if 'time_period' in silver_df.columns and 'time_period' in gold_df.columns:
        silver_timeframes = set(silver_df['time_period'].unique())
        gold_timeframes = set(gold_df['time_period'].unique())
        
        print(f"\n⏰ TIMEFRAME COVERAGE:")
        print(f"  Silver Timeframes: {sorted(silver_timeframes)}")
        print(f"  Gold Timeframes:   {sorted(gold_timeframes)}")
        
        missing_in_gold = silver_timeframes - gold_timeframes
        if missing_in_gold:
            print(f"  ⚠️  Missing in Gold: {sorted(missing_in_gold)}")
        
        extra_in_gold = gold_timeframes - silver_timeframes
        if extra_in_gold:
            print(f"  ℹ️  Extra in Gold: {sorted(extra_in_gold)}")
    
    # Column comparison
    silver_cols = set(silver_df.columns)
    gold_cols = set(gold_df.columns)
    
    print(f"\n📋 COLUMN COMPARISON:")
    print(f"  Silver Columns: {len(silver_cols)}")
    print(f"  Gold Columns:   {len(gold_cols)}")
    
    common_cols = silver_cols & gold_cols
    silver_only = silver_cols - gold_cols  
    gold_only = gold_cols - silver_cols
    
    print(f"  Common:         {len(common_cols)}")
    print(f"  Silver Only:    {len(silver_only)}")
    print(f"  Gold Only:      {len(gold_only)}")
    
    if silver_only:
        print(f"\n  📊 Silver-only columns: {sorted(silver_only)}")
    
    if gold_only:
        print(f"\n  🥇 Gold-only columns: {sorted(gold_only)}")

elif not silver_df.empty:
    print("⚠️ Only Silver data available - cannot compare with Gold")
elif not gold_df.empty:
    print("⚠️ Only Gold data available - cannot compare with Silver")
else:
    print("❌ No data available for comparison")

## 📋 Summary Report

In [ ]:
# Final Summary Report
print("📋 SOLANA DATA PIPELINE - SUMMARY REPORT")
print("="*70)

current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"🕐 Report Generated: {current_time}")

# Silver layer summary
if not silver_df.empty:
    print(f"\n✅ SILVER WALLET PNL LAYER:")
    print(f"   📊 Records: {len(silver_df):,}")
    print(f"   🏦 Unique Wallets: {silver_df['wallet_address'].nunique():,}")
    print(f"   ⏰ Timeframes: {len(silver_df['time_period'].unique())}")
    
    # PnL insights
    if 'total_pnl' in silver_df.columns:
        profitable = (silver_df['total_pnl'] > 0).sum()
        avg_pnl = silver_df['total_pnl'].mean()
        print(f"   💰 Profitable Records: {profitable:,}/{len(silver_df):,} ({profitable/len(silver_df)*100:.1f}%)")
        print(f"   📈 Average PnL: ${avg_pnl:.2f}")
    
    # Trading activity
    if 'trade_count' in silver_df.columns:
        total_trades = silver_df['trade_count'].sum()
        avg_trades = silver_df['trade_count'].mean()
        print(f"   🔄 Total Trades: {total_trades:,.0f}")
        print(f"   📊 Avg Trades/Wallet: {avg_trades:.1f}")

else:
    print(f"\n❌ SILVER LAYER: No data available")

# Gold layer summary  
if not gold_df.empty:
    print(f"\n✅ GOLD SMART WALLETS LAYER:")
    print(f"   📊 Records: {len(gold_df):,}")
    print(f"   🏦 Unique Wallets: {gold_df['wallet_address'].nunique():,}")
    
    if 'time_period' in gold_df.columns:
        print(f"   ⏰ Timeframes: {len(gold_df['time_period'].unique())}")
    
    # Classifications
    if 'trader_classification' in gold_df.columns:
        classifications = gold_df['trader_classification'].value_counts()
        print(f"   🏆 Classifications:")
        for classification, count in classifications.items():
            print(f"      {classification}: {count:,}")
    
    # Smart trader scores
    if 'smart_trader_score' in gold_df.columns:
        avg_score = gold_df['smart_trader_score'].mean()
        max_score = gold_df['smart_trader_score'].max()
        print(f"   🎯 Avg Smart Score: {avg_score:.3f}")
        print(f"   🏆 Max Smart Score: {max_score:.3f}")

else:
    print(f"\n❌ GOLD LAYER: No data available")

# Pipeline effectiveness
if not silver_df.empty and not gold_df.empty:
    silver_wallets = silver_df['wallet_address'].nunique()
    gold_wallets = gold_df['wallet_address'].nunique()
    filter_rate = ((silver_wallets - gold_wallets) / silver_wallets) * 100
    
    print(f"\n🔧 PIPELINE EFFECTIVENESS:")
    print(f"   📥 Input: {silver_wallets:,} wallets")
    print(f"   📤 Output: {gold_wallets:,} wallets")
    print(f"   🚫 Filtered: {filter_rate:.1f}%")

# Recommendations
print(f"\n💡 RECOMMENDATIONS:")

if not silver_df.empty and 'total_pnl' in silver_df.columns:
    profitable_pct = (silver_df['total_pnl'] > 0).sum() / len(silver_df) * 100
    if profitable_pct == 0:
        print(f"   ⚠️  No profitable wallets found - review PnL calculation")
    elif profitable_pct < 10:
        print(f"   ⚠️  Very low profitability ({profitable_pct:.1f}%) - check data quality")

if not silver_df.empty and not gold_df.empty:
    filter_rate = ((silver_df['wallet_address'].nunique() - gold_df['wallet_address'].nunique()) / 
                   silver_df['wallet_address'].nunique()) * 100
    if filter_rate < 5:
        print(f"   ⚠️  Low filtering rate ({filter_rate:.1f}%) - consider tightening criteria")
    
    if 'time_period' in gold_df.columns and 'time_period' in silver_df.columns:
        if len(gold_df['time_period'].unique()) < len(silver_df['time_period'].unique()):
            print(f"   ⚠️  Missing timeframes in gold layer - run DBT for all periods")

print(f"\n🔚 Analysis Complete - Data pipeline review finished")
print("="*70)

In [ ]:
# Test Updated Gold Criteria (DBT Logic)
print("🧪 TESTING UPDATED GOLD CRITERIA - DBT LOGIC")
print("="*70)

# Reconnect to test the updated criteria
conn = duckdb.connect()
conn.execute("INSTALL httpfs;")
conn.execute("LOAD httpfs;")
conn.execute("SET s3_endpoint='localhost:9000';")
conn.execute("SET s3_access_key_id='minioadmin';")
conn.execute("SET s3_secret_access_key='minioadmin123';")
conn.execute("SET s3_use_ssl=false;")
conn.execute("SET s3_url_style='path';")

# Test the exact DBT logic
print("🔍 Testing filter criteria from updated dbt model...")

try:
    # Apply the exact filtering logic from our updated dbt model
    dbt_gold_test = conn.execute("""
    WITH silver_wallet_pnl AS (
        SELECT *
        FROM read_parquet('s3://solana-data/silver/wallet_pnl/**/*.parquet')
    ),
    
    filtered_smart_wallets AS (
        SELECT 
            wallet_address,
            time_period,
            trade_count,
            win_rate,
            total_pnl,
            roi,
            realized_pnl,
            unrealized_pnl,
            total_bought,
            total_sold,
            processed_for_gold
        FROM silver_wallet_pnl
        WHERE 
            -- Focus on portfolio-level records for 'all' timeframe
            token_address = 'ALL_TOKENS' 
            AND time_period = 'all'
            
            -- Updated gold criteria matching smart_trader_config.py
            AND total_pnl >= 10.0           -- MIN_TOTAL_PNL = 10.0
            AND roi >= 1.0                  -- MIN_ROI_PERCENT = 1.0
            AND win_rate >= 40.0            -- MIN_WIN_RATE_PERCENT = 40.0
            AND trade_count >= 1            -- MIN_TRADE_COUNT = 1
            AND total_pnl > 0               -- Must be profitable
            
            -- Only process unprocessed records
            AND processed_for_gold = false
    ),
    
    tier_classification AS (
        SELECT 
            *,
            -- Performance tier classification matching smart_trader_config.py
            CASE 
                WHEN total_pnl >= 1000 AND roi >= 30 AND win_rate >= 60 AND trade_count >= 10 THEN 'elite'
                WHEN total_pnl >= 100 AND roi >= 15 AND win_rate >= 40 AND trade_count >= 5 THEN 'strong'
                ELSE 'promising'
            END as performance_tier
        FROM filtered_smart_wallets
    )
    
    SELECT 
        COUNT(*) as total_qualifying,
        COUNT(CASE WHEN performance_tier = 'elite' THEN 1 END) as elite_count,
        COUNT(CASE WHEN performance_tier = 'strong' THEN 1 END) as strong_count,
        COUNT(CASE WHEN performance_tier = 'promising' THEN 1 END) as promising_count,
        AVG(total_pnl) as avg_pnl,
        AVG(roi) as avg_roi,
        AVG(win_rate) as avg_win_rate,
        AVG(trade_count) as avg_trades
    FROM tier_classification
    """).fetchone()
    
    if dbt_gold_test:
        total, elite, strong, promising, avg_pnl, avg_roi, avg_win_rate, avg_trades = dbt_gold_test
        
        print(f"\n✅ DBT FILTER TEST RESULTS:")
        print(f"   📊 Total Qualifying: {total}")
        print(f"   🏆 Elite Tier: {elite}")
        print(f"   💪 Strong Tier: {strong}")  
        print(f"   🌟 Promising Tier: {promising}")
        print(f"   💰 Average PnL: ${avg_pnl:.2f}")
        print(f"   📈 Average ROI: {avg_roi:.2f}%")
        print(f"   🎯 Average Win Rate: {avg_win_rate:.1f}%")
        print(f"   🔄 Average Trades: {avg_trades:.1f}")
        
        if total > 0:
            print(f"\n🎉 SUCCESS: Found {total} qualifying candidates!")
            
            # Get sample records
            sample_records = conn.execute("""
            WITH silver_wallet_pnl AS (
                SELECT *
                FROM read_parquet('s3://solana-data/silver/wallet_pnl/**/*.parquet')
            ),
            
            filtered_smart_wallets AS (
                SELECT 
                    wallet_address,
                    time_period,
                    trade_count,
                    win_rate,
                    total_pnl,
                    roi,
                    realized_pnl,
                    unrealized_pnl,
                    total_bought,
                    total_sold,
                    processed_for_gold
                FROM silver_wallet_pnl
                WHERE 
                    token_address = 'ALL_TOKENS' 
                    AND time_period = 'all'
                    AND total_pnl >= 10.0
                    AND roi >= 1.0
                    AND win_rate >= 40.0
                    AND trade_count >= 1
                    AND total_pnl > 0
                    AND processed_for_gold = false
            )
            
            SELECT 
                wallet_address,
                total_pnl,
                roi,
                win_rate,
                trade_count,
                CASE 
                    WHEN total_pnl >= 1000 AND roi >= 30 AND win_rate >= 60 AND trade_count >= 10 THEN 'elite'
                    WHEN total_pnl >= 100 AND roi >= 15 AND win_rate >= 40 AND trade_count >= 5 THEN 'strong'
                    ELSE 'promising'
                END as performance_tier
            FROM filtered_smart_wallets
            ORDER BY total_pnl DESC
            LIMIT 5
            """).fetchall()
            
            print(f"\n🔝 TOP 5 QUALIFYING CANDIDATES:")
            for i, (wallet, pnl, roi, win_rate, trades, tier) in enumerate(sample_records):
                print(f"   {i+1}. {wallet[:10]}... | PnL: ${pnl:.2f} | ROI: {roi:.2f}% | Win: {win_rate:.1f}% | Trades: {trades} | Tier: {tier.upper()}")
        
        else:
            print(f"\n❌ NO QUALIFYING CANDIDATES")
            print(f"   The updated criteria are too strict or there's no unprocessed data")
            
            # Debug: check without processed_for_gold filter
            debug_test = conn.execute("""
            SELECT COUNT(*) as total_without_filter
            FROM read_parquet('s3://solana-data/silver/wallet_pnl/**/*.parquet')
            WHERE 
                token_address = 'ALL_TOKENS' 
                AND time_period = 'all'
                AND total_pnl >= 10.0
                AND roi >= 1.0
                AND win_rate >= 40.0
                AND trade_count >= 1
                AND total_pnl > 0
            """).fetchone()
            
            if debug_test:
                total_without_filter = debug_test[0]
                print(f"   💡 Debug: {total_without_filter} candidates WITHOUT processed_for_gold filter")
                if total_without_filter > 0:
                    print(f"   ➡️  Recommendation: Reset processed_for_gold flags or ignore this filter")
    
except Exception as e:
    print(f"❌ Error testing DBT logic: {e}")

print(f"\n✅ DBT Logic Test Complete")